# Test BSM model implementation
Repeat the same for normal model implementation

In [22]:
import numpy as np

In [23]:
# import
from option_models import bsm
from option_models import normal

In [24]:
### only run this when you changed the class definition
import imp
imp.reload(bsm)
imp.reload(normal)

<module 'option_models.normal' from 'C:\\Users\\Raphael\\Documents\\GitHub\\ASP.2018.Raph\\HW3\\option_models\\normal.py'>

### Price

In [25]:
# create model
bsm1 = bsm.BsmModel(0.2)

In [26]:
# price
strike = 102
spot = 100
texp = 0.25

price = bsm1.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
print(price)
assert( abs(price - 3.10628366655) < 1e-10 )

3.10628366655


### Implied vol

In [27]:
# Randomly generate spot/strike/expiry/intr/divr/cp_sign
# Then test implied volatility

for k in range(100):
    spot = np.random.uniform(80,100)
    strike = np.random.uniform(80,100)
    vol = np.random.uniform(0.0001, 0.4)
    texp = np.random.uniform(0.1, 5)
    intr = np.random.uniform(0, 0.3)
    divr = np.random.uniform(0, 0.3)
    cp_sign = 1 if np.random.rand() > 0.5 else -1

    #print( spot, strike, vol, texp, intr, divr, cp_sign)

    bsm2 = bsm.BsmModel(vol=vol, intr=intr, divr=divr)
    price = bsm2.price(strike, spot, texp, cp_sign )
    
    # get implied vol
    vol_imp = bsm2.impvol(price, strike, spot, texp=texp, cp_sign=cp_sign)
    
    # now price option with the obtained implied vol
    bsm2.vol = vol_imp
    price_imp = bsm2.price(strike, spot, texp, cp_sign )
    
    # compare the two prices
    assert( abs(price - price_imp) < 1e-8 )

AttributeError: 'BsmModel' object has no attribute 'impvol'

### Delta
Verify delta by comparing numerical derivative

In [28]:
bsm1 = bsm.BsmModel(0.2)
strike = 102
spot = 100
texp = 0.25
h=1e-6

In [29]:
delta = bsm1.delta(strike = strike, spot = spot, texp=texp, cp_sign=1)
h = 1e-6
price_up = bsm1.price(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
price_dn = bsm1.price(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
delta_numeric = ( price_up - price_dn )/(2*h)
assert( abs(delta - delta_numeric) < 1e-6 )

NameError: name 'vol' is not defined

### Vega
Do similar verification

In [ ]:
vega = bsm1.vega(strike=strike, spot=spot, vol = bsm1.vol, texp=texp, cp_sign=1)
h = 1e-6
bsm_up = bsm.BsmModel(bsm1.vol+h)
bsm_down = bsm.BsmModel(bsm1.vol-h)
price_up = bsm_up.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
price_dn = bsm_down.price(strike=strike, spot=spot, texp=texp, cp_sign=1)
vega_numeric = ( price_up - price_dn )/(2*h)
assert (abs(vega- vega_numeric) < 1e-6)

### Gamma
Do similar verification

In [ ]:
gamma = bsm1.gamma(strike=strike, spot=spot, vol = bsm1.vol, texp=texp, cp_sign=1)
h = 1e-6
delta_up = bsm1.delta(strike=strike, spot=spot+h, texp=texp, cp_sign=1)
delta_dn = bsm1.delta(strike=strike, spot=spot-h, texp=texp, cp_sign=1)
gamma_numeric = ( delta_up - delta_dn )/(2*h)
assert (abs(gamma - gamma_numeric) < 1e-6)